In [242]:
# Задание на повторение материала предыдущего семестра

In [2]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [3]:
# Генерируем уникальный seed
my_code = "Матвеичева Катя"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [4]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [5]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [6]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [7]:
example_data.shape

(2315, 7)

In [8]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [9]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [10]:
train.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
1651,2009,Magadan Oblast,-1.2,12.1,13.5,45.84,94.90
862,2000,Voronezh Oblast,-10.3,7.6,17.9,78.69,61.44
1209,2004,Kabardino-Balkar Republic,0.8,10.5,9.8,62.53,58.60
1096,2002,Republic of Khakassia,-5.8,11.2,17.0,62.33,70.88
1202,2004,Voronezh Oblast,-10.0,8.6,18.5,68.67,62.00


In [11]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [12]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [13]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [14]:
sc_val

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.407407,0.289926,0.181435,0.525346,0.508427,0.580071,Astrakhan Oblast
1,0.518519,0.135135,0.118143,0.751152,0.54318,0.498494,Kursk Oblast
2,0.592593,0.255528,0.151899,0.552995,0.429018,0.677884,Volgograd Oblast
3,0.740741,0.518428,0.35443,0.285714,0.361425,0.406835,Kabardino-Balkar Republic
4,0.814815,0.339066,0.324895,0.589862,0.6245,0.481472,Kurgan Oblast
...,...,...,...,...,...,...,...
458,0.888889,0.358722,0.265823,0.488479,0.482537,0.759068,Moscow Oblast
459,0.555556,0.486486,0.400844,0.382488,0.037011,0.543014,Chukotka Autonomous Okrug
460,0.518519,0.34398,0.223629,0.474654,0.077845,0.90965,Magadan Oblast
461,0.925926,0.341523,0.265823,0.520737,0.651607,0.812754,Kemerovo Oblast


In [15]:
# Вспоминаем алгоритмы решения задачи регрессии
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
r_models.append(SVR(kernel='linear'))
r_models.append(SVR(kernel='poly'))
r_models.append(SVR(kernel='rbf'))
r_models.append(SVR(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'mse', 'friedman_mse', 'mae', 'poisson'
r_models.append(DecisionTreeRegressor(criterion='squared_error'))
r_models.append(DecisionTreeRegressor(criterion='friedman_mse'))
r_models.append(DecisionTreeRegressor(criterion='absolute_error'))

In [16]:
r_models

[LinearRegression(),
 Lasso(),
 Ridge(),
 ElasticNet(),
 KNeighborsRegressor(),
 KNeighborsRegressor(n_neighbors=10),
 KNeighborsRegressor(n_neighbors=15),
 SVR(kernel='linear'),
 SVR(kernel='poly'),
 SVR(),
 SVR(kernel='sigmoid'),
 DecisionTreeRegressor(),
 DecisionTreeRegressor(criterion='friedman_mse'),
 DecisionTreeRegressor(criterion='absolute_error')]

In [17]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [18]:
x_labels

['year', 'npg', 'birth_rate', 'death_rate', 'gdw']

In [19]:
y_labels

['urbanization']

In [20]:
# Обучаем модели
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

LinearRegression()
Lasso()
Ridge()
ElasticNet()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [21]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

LinearRegression() 	 0.019046783730023175
Lasso() 	 0.028988166491357544
Ridge() 	 0.01903247307926995
ElasticNet() 	 0.028988166491357544
KNeighborsRegressor() 	 0.010172006305084975
KNeighborsRegressor(n_neighbors=10) 	 0.011114039027231064
KNeighborsRegressor(n_neighbors=15) 	 0.012737493143109157
SVR(kernel='linear') 	 0.019300823133766704
SVR(kernel='poly') 	 0.012777721131284971
SVR() 	 0.011320760215086958
SVR(kernel='sigmoid') 	 32.79293981788303
DecisionTreeRegressor() 	 0.019500846417711073
DecisionTreeRegressor(criterion='friedman_mse') 	 0.020117364546145596
DecisionTreeRegressor(criterion='absolute_error') 	 0.021245077258325463


In [22]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [23]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.011798482751493578


In [24]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l1', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='linear'))
c_models.append(SVC(kernel='poly'))
c_models.append(SVC(kernel='rbf'))
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='random'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='random'))

In [25]:
c_models

[LogisticRegression(max_iter=500, penalty='none', solver='saga'),
 LogisticRegression(max_iter=500, penalty='l1', solver='saga'),
 LogisticRegression(solver='saga'),
 LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'),
 LogisticRegression(),
 MultinomialNB(alpha=0.0),
 MultinomialNB(alpha=0.5),
 MultinomialNB(),
 KNeighborsClassifier(),
 KNeighborsClassifier(n_neighbors=10),
 KNeighborsClassifier(n_neighbors=15),
 SVC(kernel='linear'),
 SVC(kernel='poly'),
 SVC(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random'),
 DecisionTreeClassifier(criterion='entropy'),
 DecisionTreeClassifier(criterion='entropy', splitter='random')]

In [26]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [27]:
x_train

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.703704,0.361179,0.248945,0.474654,0.144917,0.93322
1,0.37037,0.137592,0.059072,0.677419,0.715725,0.49509
2,0.518519,0.410319,0.181435,0.304147,0.434926,0.457902
3,0.444444,0.248157,0.21097,0.635945,0.431451,0.618698
4,0.518519,0.144963,0.101266,0.705069,0.541616,0.502422
...,...,...,...,...,...,...
1384,0.851852,0.398034,0.291139,0.442396,0.570808,0.391122
1385,0.333333,0.321867,0.160338,0.442396,0.707732,0.528873
1386,1.0,0.275184,0.181435,0.552995,0.78384,0.760377
1387,0.407407,0.191646,0.075949,0.59447,0.401738,0.706298


In [28]:
# Обучаем модели
for model in c_models:
    print(model)
    model.fit(x_train, y_train)

LogisticRegression(max_iter=500, penalty='none', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(max_iter=500, penalty='l1', solver='saga')
LogisticRegression(solver='saga')
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')
LogisticRegression()
MultinomialNB(alpha=0.0)
MultinomialNB(alpha=0.5)
MultinomialNB()
KNeighborsClassifier()
KNeighborsClassifier(n_neighbors=10)


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


KNeighborsClassifier(n_neighbors=15)
SVC(kernel='linear')
SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')
DecisionTreeClassifier(criterion='entropy')
DecisionTreeClassifier(criterion='entropy', splitter='random')


In [29]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

LogisticRegression(max_iter=500, penalty='none', solver='saga') 	 0.3428863454489997
LogisticRegression(max_iter=500, penalty='l1', solver='saga') 	 0.11904218292685943
LogisticRegression(solver='saga') 	 0.06975812742589216
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga') 	 0.08811956572401271
LogisticRegression() 	 0.06975812742589216
MultinomialNB(alpha=0.0) 	 0.025566256129311914
MultinomialNB(alpha=0.5) 	 0.025194151153760124
MultinomialNB() 	 0.025441756222805412
KNeighborsClassifier() 	 0.3466320296835713
KNeighborsClassifier(n_neighbors=10) 	 0.27674769807516325
KNeighborsClassifier(n_neighbors=15) 	 0.22784986924748177
SVC(kernel='linear') 	 0.08377708695837577
SVC(kernel='poly') 	 0.6692691622747193
SVC() 	 0.20735471911115205
SVC(kernel='sigmoid') 	 0.00037562212414311207
DecisionTreeClassifier() 	 0.6165485552086264
DecisionTreeClassifier(splitter='random') 	 0.5600846213316039
DecisionTreeClassifier(criterion='entropy') 	 0.5755378492531533
DecisionTre

In [30]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

SVC(kernel='poly')

In [31]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.6828130085022476


In [32]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [33]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.703704,0.361179,0.248945,0.474654,0.144917,0.93322
1,0.37037,0.137592,0.059072,0.677419,0.715725,0.49509
2,0.518519,0.410319,0.181435,0.304147,0.434926,0.457902
3,0.444444,0.248157,0.21097,0.635945,0.431451,0.618698
4,0.518519,0.144963,0.101266,0.705069,0.541616,0.502422
...,...,...,...,...,...,...
458,0.925926,0.707617,0.510549,0.101382,0.439791,0.281131
459,0.592593,0.351351,0.324895,0.571429,0.32841,0.53123
460,0.074074,0.280098,0.118143,0.474654,0.757602,0.703418
461,0.888889,0.439803,0.316456,0.391705,0.446047,0.628126


In [34]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [35]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.2953979292128132
0.25434907606349066
0.23314818501912982
0.23427508148694062


In [36]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=5)
0.2953979292128132


In [54]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25),
    SVR(kernel='linear'),
    SVR(kernel='poly'),
    SVR(kernel='rbf'),
    SVR(kernel='sigmoid'),
    DecisionTreeRegressor(criterion='squared_error'),
    DecisionTreeRegressor(criterion='friedman_mse'),
    DecisionTreeRegressor(criterion='absolute_error')
]

In [55]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[Ridge(), SVR(), ElasticNet(alpha=0.5), ElasticNet(alpha=0.5, l1_ratio=0.75)]


In [56]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv", decimal=',') 
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,-3.0,-0.3,-0.4
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,-2.0,0.6,-1.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,-7.0,-1.8,-0.3
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,-1.0,1.2,-1.3
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,-2.0,2.5,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,-1.0,5.9,1.5
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,-1.0,5.2,3.1
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,1.0,1.4,1.9
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,-4.0,2.7,1.7


In [57]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = 'water_level'
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['visibility', 'wind', 'humidity', 'pressure', 'wl_change']


In [58]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

data = data.drop(['precipitation', 'temperature', 'fire', 'temp_change', 'pressure_change', 'weather'],axis=1)


In [59]:
data

,water_level,humidity,visibility,wind,pressure,wl_change
0,468.0,31.0,100.0,4.0,993.8,-3.0
1,140.0,22.0,20.0,1.0,1004.1,-2.0
2,464.0,28.0,20.0,2.0,1001.2,-7.0
3,467.0,26.0,100.0,3.0,992.3,-1.0
4,138.0,22.0,20.0,1.0,1001.3,-2.0
...,...,...,...,...,...,...
1434,474.0,95.0,4.0,1.0,999.9,-1.0
1435,65.0,93.0,10.0,4.0,983.9,-1.0
1436,159.0,83.0,10.0,2.0,969.7,1.0
1437,99.0,91.0,10.0,3.0,995.4,-4.0


In [60]:
data.shape
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [61]:
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [62]:
train.head()

,water_level,humidity,visibility,wind,pressure,wl_change
1030,126.0,95.0,4.0,1.0,964.8,-1.0
1072,82.0,58.0,20.0,3.0,984.0,-2.0
1130,300.0,84.0,20.0,5.0,991.5,2.0
781,526.0,78.0,100.0,3.0,963.0,6.0
169,143.0,96.0,0.0,1.0,988.1,18.0


In [63]:
columns_to_scale = ['visibility', 'wind', 'humidity', 'pressure', 'wl_change']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['visibility', 'wind', 'humidity', 'pressure',
                                  'wl_change'])])

In [64]:
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))
column_names = columns_to_scale + ['water_level']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names
sc_val

,visibility,wind,humidity,pressure,wl_change,water_level
0,0.04,0.0,0.912088,0.755656,0.383495,451.0
1,0.10,0.0,0.923077,0.644796,0.388350,211.0
2,0.10,0.0,0.868132,0.653846,0.344660,41.0
3,0.20,0.1,0.769231,0.767722,0.388350,343.0
4,0.20,0.3,0.263736,0.664404,0.495146,293.0
...,...,...,...,...,...,...
283,0.20,0.3,0.472527,0.773756,0.320388,324.0
284,0.20,0.1,0.758242,0.789593,0.383495,78.0
285,0.20,0.5,0.395604,0.576169,0.373786,41.0
286,0.20,0.4,0.263736,0.720965,0.276699,389.0


In [65]:
r_models = []

r_models.append(ElasticNet(alpha=0.5))
r_models.append(ElasticNet(alpha=0.5, l1_ratio=0.75))

r_models.append(SVR())

r_models.append(Ridge())
r_models

[ElasticNet(alpha=0.5), ElasticNet(alpha=0.5, l1_ratio=0.75), SVR(), Ridge()]

In [66]:
x_labels = column_names[0:-1]
y_labels = ['water_level']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [67]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

ElasticNet(alpha=0.5)
ElasticNet(alpha=0.5, l1_ratio=0.75)
SVR()
Ridge()


In [68]:
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

ElasticNet(alpha=0.5) 	 54359.24539331417
ElasticNet(alpha=0.5, l1_ratio=0.75) 	 53923.04627085828
SVR() 	 58601.12053478997
Ridge() 	 53282.813978524086


In [69]:
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

Ridge()

In [70]:
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

43677.709903250696


In [71]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25),
    SVC(kernel='linear'),
    SVC(kernel='poly'),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid'),
    DecisionTreeClassifier(criterion='gini', splitter='best'),
    DecisionTreeClassifier(criterion='gini', splitter='random'),
    DecisionTreeClassifier(criterion='entropy', splitter='best'),
    DecisionTreeClassifier(criterion='entropy', splitter='random')
]

In [72]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[LogisticRegression(penalty='l1', solver='saga'), KNeighborsClassifier(), DecisionTreeClassifier(criterion='entropy'), DecisionTreeClassifier(splitter='random'), SVC(kernel='sigmoid')]


In [73]:
# Загрузим данные для задачи классификации
newdata = pd.read_csv("datasets/zoo2.csv")
newdata

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [74]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(newdata.columns)
n_x = 8

y_newlabel = 'class_type'
x_newlabels = random.sample(columns[1:-1], n_x)

print(x_newlabels)

['fins', 'tail', 'venomous', 'catsize', 'legs', 'backbone', 'toothed', 'breathes']


In [75]:
labels = x_newlabels.copy()
labels.append(y_newlabel)
temp_data = newdata[labels]
sec_data = temp_data

In [76]:
sec_data

,fins,tail,venomous,catsize,legs,backbone,toothed,breathes,class_type
0,0,1,0,1,4,1,0,1,3
1,0,1,0,0,4,1,1,1,3
2,0,1,0,1,4,1,1,1,3
3,0,1,0,0,4,1,1,1,3
4,0,1,0,0,4,1,1,1,3
5,0,1,1,1,0,1,1,1,3
6,0,1,0,1,0,1,1,1,3
7,0,1,1,1,0,1,1,1,3
8,0,1,0,1,4,1,1,1,3
9,0,1,0,1,4,1,1,1,3


In [77]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

sec_data.shape
val_test_size2 = round(0.2*len(sec_data))
print(val_test_size2)

9


In [78]:
random_state = my_seed
train_val2, test2 = train_test_split(sec_data, test_size=val_test_size2, random_state=random_state)
sec_train, val2 = train_test_split(train_val2, test_size=val_test_size2, random_state=random_state)
print(len(train2), len(val2), len(test2))

NameError: name 'train2' is not defined

In [79]:
sec_train.head()

,fins,tail,venomous,catsize,legs,backbone,toothed,breathes,class_type
24,0,0,0,0,4,1,1,1,5
8,0,1,0,1,4,1,1,1,3
7,0,1,1,1,0,1,1,1,3
3,0,1,0,0,4,1,1,1,3
23,0,0,0,0,4,1,1,1,5


In [80]:
columns_to_scale2 = x_newlabels

ct2 = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale2)], remainder='passthrough')
ct2.fit(sec_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['fins', 'tail', 'venomous', 'catsize', 'legs',
                                  'backbone', 'toothed', 'breathes'])])

In [81]:
sc_train2 = pd.DataFrame(ct2.transform(sec_train))
sc_test2 = pd.DataFrame(ct2.transform(test2))
sc_val2 = pd.DataFrame(ct2.transform(val2))
column_names2 = columns_to_scale2 + ['class_type']
sc_train2.columns = column_names2
sc_test2.columns = column_names2
sc_val2.columns = column_names2
sc_val2

,fins,tail,venomous,catsize,legs,backbone,toothed,breathes,class_type
0,0.0,0.0,0.0,0.0,0.75,0.0,0.0,1.0,6.0
1,1.0,1.0,0.0,1.0,0.00,1.0,0.0,0.0,4.0
2,1.0,1.0,0.0,1.0,0.00,1.0,0.0,0.0,4.0
3,0.0,1.0,0.0,0.0,0.50,1.0,1.0,1.0,3.0
4,0.0,0.0,0.0,0.0,0.75,0.0,0.0,1.0,6.0
5,0.0,1.0,0.0,1.0,0.50,1.0,0.0,1.0,3.0
6,1.0,1.0,0.0,1.0,0.00,1.0,0.0,0.0,4.0
7,0.0,1.0,0.0,1.0,0.50,1.0,1.0,1.0,3.0
8,0.0,1.0,0.0,0.0,0.25,1.0,1.0,1.0,5.0


In [82]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='l1', solver='saga'))


# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier())
c_models.append(DecisionTreeClassifier(splitter='random'))
c_models
#[MultinomialNB(), KNeighborsClassifier, DecisionTreeClassifier(), LogisticRegression, KNeighborsClassifier]

[LogisticRegression(penalty='l1', solver='saga'),
 KNeighborsClassifier(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random')]

In [83]:
x_labels2 = column_names2[0:-1]
y_labels2 = ['class_type']

x_train2 = sc_train2[x_labels2]
x_test2 = sc_test2[x_labels2]
x_val2 = sc_val2[x_labels2]

y_train2 = np.ravel(sc_train2[y_labels2])
y_test2 = np.ravel(sc_test2[y_labels2])
y_val2 = np.ravel(sc_val2[y_labels2])
x_train2

,fins,tail,venomous,catsize,legs,backbone,toothed,breathes
0,0.0,0.0,0.0,0.0,0.50,1.0,1.0,1.0
1,0.0,1.0,0.0,1.0,0.50,1.0,1.0,1.0
2,0.0,1.0,1.0,1.0,0.00,1.0,1.0,1.0
3,0.0,1.0,0.0,0.0,0.50,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.50,1.0,1.0,1.0
5,0.0,1.0,0.0,1.0,0.00,1.0,1.0,1.0
6,0.0,1.0,0.0,0.0,0.50,1.0,1.0,1.0
7,0.0,0.0,0.0,0.0,0.75,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.50,1.0,1.0,1.0
9,0.0,0.0,0.0,0.0,0.75,0.0,0.0,1.0


In [84]:
# Обучаем модели
for model2 in c_models:
    print(model2)
    model2.fit(x_train2, y_train2)

LogisticRegression(penalty='l1', solver='saga')
KNeighborsClassifier()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [85]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model2 in c_models:
    val_pred2 = model2.predict(x_val2)
    f1 = f1_score(y_val2, val_pred2, average='weighted')
    f1s.append(f1)
    print(model2, '\t', f1)

LogisticRegression(penalty='l1', solver='saga') 	 0.8412698412698413
KNeighborsClassifier() 	 0.4222222222222222
SVC(kernel='sigmoid') 	 0.4222222222222222
DecisionTreeClassifier() 	 0.7777777777777778
DecisionTreeClassifier(splitter='random') 	 0.8412698412698413


In [86]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

LogisticRegression(penalty='l1', solver='saga')

In [87]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred2 = best_c_model.predict(x_test2)
f1 = f1_score(y_test2, test_pred2, average='weighted')
print(f1)

0.7037037037037037


In [88]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=40))

In [89]:
# Выделим используемые параметры
x_labels2 = column_names2[0:-1]
x_data2 = pd.concat([sc_train2[x_labels2], sc_val2[x_labels2], sc_test2[x_labels2]])
x_data2

,fins,tail,venomous,catsize,legs,backbone,toothed,breathes
0,0.0,0.0,0.0,0.0,0.50,1.0,1.0,1.0
1,0.0,1.0,0.0,1.0,0.50,1.0,1.0,1.0
2,0.0,1.0,1.0,1.0,0.00,1.0,1.0,1.0
3,0.0,1.0,0.0,0.0,0.50,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.50,1.0,1.0,1.0
5,0.0,1.0,0.0,1.0,0.00,1.0,1.0,1.0
6,0.0,1.0,0.0,0.0,0.50,1.0,1.0,1.0
7,0.0,0.0,0.0,0.0,0.75,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.50,1.0,1.0,1.0
9,0.0,0.0,0.0,0.0,0.75,0.0,0.0,1.0


In [90]:
# Произведем кластеризацию
for model2 in k_models:
    model2.fit(x_data2)

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (17) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:3: ConvergenceWarning: Number of distinct clusters (17) found smaller than n_clusters (40). Possibly due to duplicate points in X.
  This is separate from the ipykernel package so we can avoid doing imports until


In [91]:
# Оценим качество результата
sils = []
for model2 in k_models:
    cluster_labels = model2.predict(x_data2)
    s = silhouette_score(x_data2, cluster_labels)
    sils.append(s)
    print(s)

0.5896174574421572
0.7191948080283129
0.7674418604651163
0.7674418604651163


In [92]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=20)
0.7674418604651163
